## Code to convert geosoft grids to geotiffs based on info at:

https://help.seequent.com/Oasis-montaj/9.9/en/Content/ss/process_data/grid_data/c/geosoft_grid_file_format.htm plus info that compression is in fact zlib not LZRW1 regardless of what COMP_TYPE says!

Doesn't handle everything yet:      
- Should check data type in ES & SF and handle data types accordingly
- Could write tif->grd as well?
- Could parse XML if present for CRS info
- Could parse .gi file for CRS and other info if I had format...
   
Test data from CPRM Open Access geophysical data server https://geosgb.cprm.gov.br/geosgb/downloads.html

## Parse geosoft grid

In [ ]:
import struct
import numpy as np
import zlib
import os

def parse_geosoft_grid(file_path,epsg):
#read geosoft binary grid and return components
#inputs:
    #file_path: path to geosoft grid (str)
    #epsg: EPSG projection ID (int)
#returns:
    #header: header data (dict)
    #grid: grid data (2D array of float32)
    
    
    #header data sizes
    lengths=np.array([ 4,   4,  4,  4,  4,  8,  8,  8,  8,  8,  8,  8,  48, 16, 
                       4,  4,  4,  4,  4,  4,  4,  4,  4,  8,  4,  
                       324,4,  4,  4,  4]) 
    #header data types
    formats=np.array(['i','i','i','i','i','d','d','d','d','d','d','d','s','s',
                      'i','i','i','i','i','f','f','f','f','d','i',
                      's','s','i','i','i'])
    #header variable names
    labels=np.array(['ES','SF','NE','NV','KX','DE','DV','X0','Y0','ROT','ZBASE','ZMULT','LABEL','MAPNO',
                     'PROJ','UNITX','UNITY','UNITZ','NVPTS','IZMIN','IZMAX','IZMED','IZMEA','ZVAR','PRCS',
                     'USER','SIG','COMP_TYPE','NB','VPB'])
    #load header
    file=open(file_path,'rb')
    header={}
    for i in range(len(lengths)-4): #last 4 header items only if compressed
        
        val=file.read(lengths[i])
        
        if(formats[i]=='s'):
            (val_conv,)=struct.unpack(str(lengths[i])+formats[i],val)
        else:
            (val_conv,)=struct.unpack(formats[i],val)
        header[labels[i]]=val_conv
    
    #load grid
    if(header['ES']>8): #compressed grid

        #load compressed file extra header
        for j in range(i+1,i+5):

            val=file.read(lengths[j])

            if(formats[j]=='s'):
                (val_conv,)=struct.unpack(str(lengths[j])+formats[j],val)
            else:
                (val_conv,)=struct.unpack(formats[j],val)
            header[labels[j]]=val_conv
        OBS=struct.unpack(str(header['NB'])+'q',file.read(8*header['NB'])) # File offset from start of every block
        CBS=struct.unpack(str(header['NB'])+'i',file.read(4*header['NB'])) # Compressed size of every block
        
        file=open(file_path,'rb')
        bgrid=b''
        for i in range(header['NB']):
            file.seek(OBS[i]+16,0)    #unexplained 16 byte header
            data=file.read(CBS[i]-16) #unexplained 16 byte header

            result=zlib.decompress(data, bufsize=zlib.DEF_BUF_SIZE)
            bgrid=bgrid+result
            
        grid=np.frombuffer( bgrid, np.float32 )
        grid=np.where(grid < -1e8, -1e8, grid)
        grid=grid.reshape((header['NV'],header['NE']))
        
    else: #uncompressed grid (zlib compression with 16 byte header ignored) 

        grid=np.fromfile(file_path, dtype=np.float32, offset=512) #should check data type in ES & SF and handle
        grid=np.where(grid < -1e8, -1e8, grid)
        grid=grid.reshape((header['NV'],header['NE']))

    header['PROJ']=epsg # could read from xml if present, info is stored in wellknown_epsg
    
    
    return(header,grid)
    


## Save out grid as geotiff

In [ ]:
import rasterio
from rasterio.transform import from_origin

def write_geotiff(out_path,header,grid):
    
    transform = from_origin(header['X0']-(header['DE']/2), header['Y0']-(header['DV']/2), header['DE'], -header['DV'])

    new_dataset = rasterio.open(
        out_path,
        "w",
        driver="GTiff",
        height=header['NV'],
        width=header['NE'],
        count=1,
        dtype=rasterio.float32,
        crs='epsg:'+str(header['PROJ']),#"+proj=longlat",
        transform=transform,
        nodata=-1e8,
    )

    new_dataset.write(grid, 1)
    new_dataset.close()
    print("dtm geotif saved as", out_path)


def convert_geosoft_grid_to_tif(filename,in_path,out_path,epsg):
    header,grid=parse_geosoft_grid(os.path.join(in_path, filename),epsg)
    filename=filename.replace('.grd','.tif')
    if(!os.path.isdir(out_path)):
        os.mkdir(out_path) 
    write_geotiff(os.path.join(out_path, filename),header,grid) 


In [ ]:
in_path='./test_data'              
out_path='./output_data'

filename='test_uncompressed.grd' # uncompressed example
#filename='test_compressed.grd'   # compressed example

convert_geosoft_grid_to_tif(filename,in_path,out_path,4326)